<a href="https://colab.research.google.com/github/Ahtesham519/Praticle_nlp_book/blob/main/nert_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install sklearn-crfsuite


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 24.8 MB/s eta 0:00:00


In [3]:
from nltk.tag import pos_tag
from sklearn_crfsuite import CRF , metrics
from sklearn.metrics import make_scorer , confusion_matrix
from pprint import pprint
from sklearn.metrics import f1_score , classification_report
from sklearn.pipeline import Pipeline
import string
import warnings
warnings.filterwarnings('ignore')


Loading the dataset

In [4]:
def load_data_conll(file_path):
  myoutput , words, tags = [] , [] , []
  fh = open(file_path)
  for line in fh:
    line= line.strip()
    if "\t" not in line:
      #Sentences ended.
      myoutput.append([words, tags])
      words , tags = [] , []
    else:
      word , tag = line.split("\t")
      words.append(word)
      tags.append(tag)
  fh.close()
  return myoutput

In [5]:
def sent2feats(sentence):
  feats = []
  sen_tags = pos_tag(sentence)  #This format is specific to this POS tagger!
  for i in range(0 ,len(sentence)):
    word = sentence[i]
    wordfeats = {}
    #word features: word , prev 2 words , next 2 words in the sentence.
    wordfeats['word'] = word
    if i == 0:
      wordfeats["prevWord"] = wordfeats ["prevSecondWord"] = "<S>" 
    elif i ==1:
      wordfeats["prevWord"] = sentence[i-1]
      wordfeats["prevSecondWord"] = sentence[i-2]
    #next two words as features 
    if i == len(sentence) -2:
      wordfeats["nextWord"]  = sentence [ i+1]
      wordfeats["nextNextWord"] = "</S>"
    elif i == len(sentence) -1:
      wordfeats["nextWords"] = "</S>"
      wordfeats["nextNextWord"] = "</S>"
    else:
      wordfeats["nextWord"] = sentence[i+1]
      wordfeats["nextNextWord"] = sentence[i+2]

    
    #POS tag features: current tag , previous and next 2 tags
    wordfeats['tag']  = sen_tags[i][1]
    if i == 0:
      wordfeats["prevTag"] =wordfeats["prevSecondWord"] = "<S>"
    elif i == 1:
      wordfeats["prevTag"] = sen_tags[0][1]
      wordfeats["prevSecondTag"] = "</S>"
    else:
      wordfeats["prevTag"]  = sen_tags[i -1][1]
      wordfeats["prevSecondTag"] = sen_tags[i - 2][1]

    #Next twon words as features

    if i == len(sentence) - 2:
      wordfeats["nextTag"] = sen_tags[i +1][1]
      wordfeats["nextNextTag"] = "</S>"
    elif i ==len(sentence) -1:
      wordfeats["nextTag"] = "</S>"
      wordfeats["nextNextTag"] = "</S>"
    else: 
      wordfeats["nextTag"] = sen_tags[i +1][1]
      wordfeats["nextNextWord"] = sen_tags [i+ 2] [1]

    #Taht is it : You can add whatever you watn!
    feats.append(wordfeats)

  return feats

Extraction Features


In [6]:
#Extract fatures from the conll data , afer loading it .

def get_feats_conll(conll_data):
  feats = []
  labels = []
  for sentence in conll_data:
    feats.append(sent2feats(sentence[0]))
    labels.append(sentence[1])
  return feats , labels
  

Training a Model

In [7]:
#Train a sequence Model 
def train_seq(X_train , Y_train , X_dev , Y_dev):
  crf = CRF(algorithm = 'lbfgs' , c1 = 0.1 , c2 =10 , max_iterations = 50) #all_posible_states = True
  crf.fit(X_train , Y_train)
  labels = list(crf.classes_)
  #testing:
  y_pred = crf.predict(X_dev)
  sorted_labels = sorted(labels, key = lambda name:(name[1:] , name[0]))
  print(metrics.flat_f1_score(Y_dev , y_pred, average= 'weighted' , labels=labels))
  print(metrics.flat_classification_report(Y_dev , y_pred , labels = sorted_labels , digits=3))

  get_confusion_matrix(Y_dev , y_pred, labels = sorted_labels)
  

In [9]:
def print_cm(cm, labels):
  print("\n")
  """ pretty print for confusion matrixes"""
  columnwidth = max([len(x) for x in labels] + [5] )   #5 is value length
  empty_cell = " " * columnwidth

  #print header
  print(" " +empty_cell , end = " ")
  for label in labels:
    print("%{0}s" .format(columnwidth) %label , end=" ")
  print()
  #print rows
  for i, label1 in enumerate(labels):
    print("   %{0}s".format(columnwidth) %label1 , end = " ")
    sum = 0
    for j in range(len(labels)):
      cell = "%{0}.0f".format(columnwidth) % cm[i,j]
      sum = sum +int(cell)
      print(cell , end ="  ")
    print(sum)  #Prints the total number of instances per cat at the end 

In [10]:
#python crfsuite does not have a confusion matrix function
#so writing it using sklearn's confusion matrix and print_cm from github

def get_confusion_matrix(y_true , y_pred , labels):
  trues, preds = [] , []
  for yseq_true , yseq_pred in zip(y_true  , y_pred):
    trues.extend(yseq_true)
    preds.extend(yseq_pred)
  print_cm(confusion_matrix(trues , preds, labels), labels) 

Call out function which are inside the main method 


In [ ]:
def main():
  try:
    from google.colab import files
    uploaded = files.upload()
    train_path = 'train.txt'
    test_path = 'test.txt'

  except:
    train_path = 'Data/conlldata/train.txt'
    test_path = 'Data/conlldata/test.txt'
  
  conll_train = load_data_conll(train_path)
  conll_dev = load_data_conll(test_path)


  print("Training a Sequence classification model with CRF")
  feats, labels = get_feats_conll(conll_train)
  devfeats , devlabels = get_feats_conll(conll_dev)
  train_seq(feats , labels , devfeats , devlabels)
  print("Done with sequence model")

if __name__ =="__main__":
  main()